In [1]:
import csv
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from sklearn import linear_model
from sklearn import tree
import math
import os
import time
from IPython.display import clear_output
import pickle

In [2]:
path="/Pickles"
pickleFolder=os.getcwd()+path

In [3]:
dataAtck=pd.read_pickle(pickleFolder+"/BaseAttackDataframe.pkl")
dataNorm=pd.read_pickle(pickleFolder+"/BaseNormalDataframe.pkl")

# 2

In [4]:
actuator_list=["MV101","P101","P102","MV201","P201","P202","P203","P204","P205","P206","MV301","MV302","MV303","MV304","P301","P302","P401","P402","P403","P404","P501","P502","P601","P602","P603"]

In [5]:
active_actuator_list=["MV101","P101","MV201","P203","P205","MV301","MV302","MV303","MV304","P301","P302","P402","P501","P602"]

In [6]:
inactive_actuator_list=["P102","P201","P202","P204","P206","P401","P403","P404","P502","P601","P603"]

In [7]:
# General regression dataframe pickles

dataAtck_reg_adpt=pd.read_pickle(pickleFolder+"/GeneralRegressionDataframeAttack.pkl")
dataNorm_reg_adpt=pd.read_pickle(pickleFolder+"/GeneralRegressionDataframeNormal.pkl")

In [8]:
# Linear regression dataframe pickles

AY_linreg=pd.read_pickle(pickleFolder+"/LinearRegressionDataframeAttack.pkl")
Y_linreg=pd.read_pickle(pickleFolder+"/LinearRegressionDataframeNormal.pkl")

In [ ]:
Y_linreg

# Logistic regression

In [9]:
from sklearn.linear_model import LogisticRegression
with open(pickleFolder+'/new_d_log.pickle', 'rb') as handle:
    d_log = pickle.load(handle)

/home/sharding/miniconda3/lib/python3.7/site-packages/sklearn/base.py:306: UserWarning: Trying to unpickle estimator LogisticRegression from version 0.20.2 when using version 0.21.2. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)


In [11]:
p_value_log=pd.DataFrame([],columns=active_actuator_list)
for p in active_actuator_list:
    try:
        kasia=np.multiply(np.array([np.array([0,1,2])==x for x in dataAtck_reg_adpt[p]]),d_log["classifier_{0}".format(p)].predict_proba(dataAtck_reg_adpt.drop([p],axis=1)))
        p_value_log[p]=sum(np.transpose(kasia))
    except:
        kasia=np.multiply(np.array([np.array([0,1])==x for x in dataAtck_reg_adpt[p]]),d_log["classifier_{0}".format(p)].predict_proba(dataAtck_reg_adpt.drop([p],axis=1)))
        p_value_log[p]=sum(np.transpose(kasia))
    print(p)

MV101
P101
MV201
P203
P205
MV301
MV302
MV303
MV304
P301
P302
P402
P501
P602


In [ ]:
pvalue_dataframe=pd.DataFrame([],columns=actuator_list)
for x in active_actuator_list:
    A=np.array(p_value_log[x])
    B=np.multiply(d_log["classifier_{0}".format(x)].predict_proba(dataAtck_reg_adpt.drop([x],axis=1)),d_log["classifier_{0}".format(x)].predict_proba(dataAtck_reg_adpt.drop([x],axis=1))<A.reshape(len(A),1))
    C=sum(np.transpose(B))
    D=C+0.5*A
    if not all(D>0):
        D=D+0.000000001
    pvalue_dataframe[x]=D
Q=np.array([0.5,0.000000001])
for x in inactive_actuator_list:
    pvalue_dataframe[x]=Q[dataAtck_reg_adpt[x]]

# Combine P-values

In [ ]:
combinedValue=np.array([])
for x in pvalue_dataframe.values:
    combinedValue=np.hstack((combinedValue,sum(np.log(x))))

In [ ]:
pvalue_dataframe["Combined"]=-2*combinedValue

In [ ]:
pvalue_dataframe_combineTime=pvalue_dataframe.copy()
#for t in range(0,len(pvalue_dataframe["Combined"])):
#    pvalue_dataframe_combineTime["Combined"][t]=sum(pvalue_dataframe["Combined"][max(0,t-20):t])

In [ ]:
h=open("/home/sharding/Documents/SummerProject/Data/AttackList_Indexed.csv","r+")
file_atcklist=csv.reader(h)
top=next(file_atcklist)
attack_list=pd.DataFrame(file_atcklist)

In [ ]:
base=time.strptime('12/28/2015 10:00:00', '%m/%d/%Y %H:%M:%S')
attack_start_times=[]
for t in range(0,len(attack_list)):
    attack_start_times.append(int(time.mktime(time.strptime(attack_list.iloc[t][1], '%m/%d/%Y %H:%M:%S'))-time.mktime(base)))

In [ ]:
h=open("/home/sharding/Documents/SummerProject/Data/AttackList_Indexed.csv","r+")
file_atcklist=csv.reader(h)
top=next(file_atcklist)
attack_list=pd.DataFrame(file_atcklist)

In [ ]:
tail=120
base=time.strptime('12/28/2015 10:00:00', '%m/%d/%Y %H:%M:%S')
attack_start_times=[]
for t in range(0,len(attack_list)):
    attack_start_times.append(int(time.mktime(time.strptime(attack_list.iloc[t][1], '%m/%d/%Y %H:%M:%S'))-time.mktime(base)))
attack_start_times=np.array(attack_start_times)
attack_end_times=[]
for t in range(0,len(attack_list)):
    attack_end_times.append(int(time.mktime(time.strptime(attack_list.iloc[t][2], '%m/%d/%Y %H:%M:%S'))-time.mktime(base)+tail))
attack_end_times=np.array(attack_end_times)
attack_bins=np.transpose(np.vstack((attack_start_times,attack_end_times)))

In [ ]:
attack_pvalues=pvalue_dataframe_combineTime.copy()
attack_pvalues["Time"]=np.array([int(time.mktime(time.strptime(t, " %d/%m/%Y %I:%M:%S %p"))-time.mktime(base)) for t in np.array(dataAtck["Timestamp"][1:])])

In [ ]:
no_attack_true_false=(dataAtck["Normal/Attack"]=="Normal")[:-1]

In [ ]:
TP_array=np.array([])
FN_array=np.array([])
FP_array=np.array([])
TN_array=np.array([])

no_attack_time_list=range(0,np.array(attack_pvalues["Time"])[-1]+1)
for x in attack_bins:
    no_attack_time_list=[t for t in no_attack_time_list if t<x[0] or t>x[1]]
num_no_attacks=len(no_attack_time_list)
no_attack_dataset=attack_pvalues[no_attack_true_false]

for p in np.arange(0,6000,10):
    TP=0
    FN=0
    for x in attack_bins:
        bin_dataset=attack_pvalues["Combined"][np.multiply(np.array(attack_pvalues["Time"]>=x[0]),np.array(attack_pvalues["Time"]<=x[1]))]
        if all(bin_dataset<p)==True:
            FN=FN+1
        else:
            TP=TP+1
    TP_array=np.hstack((TP_array,TP))
    FN_array=np.hstack((FN_array,FN))
    
    no_attack_pvalues=no_attack_dataset["Combined"]
    FP=sum(np.array(no_attack_pvalues>p))
    TN=num_no_attacks-FP
    FP_array=np.hstack((FP_array,FP))
    TN_array=np.hstack((TN_array,TN))
    clear_output()
    print(p)

In [ ]:
conditionPositive=TP_array+FN_array
conditionNegative=TN_array+FP_array

In [ ]:
TPR_array=TP_array/conditionPositive
FPR_array=FP_array/conditionNegative

In [ ]:
#with open(pickleFolder+'/FPR_LOG_FISH.pickle', 'wb') as handle:
#    pickle.dump(FPR_array,handle)
#with open(pickleFolder+'/TPR_LOG_FISH.pickle', 'wb') as handle:
#    pickle.dump(TPR_array,handle)

In [ ]:
plt.figure(figsize=(20,10))
for m in attack_start_times:
    plt.axvline(x=m,color="r",linestyle="-")
twenty_sec=np.arange(0,no_attack_time_list[-1],20)
plt.plot(attack_pvalues["Time"],pvalue_dataframe_combineTime["Combined"],".b")
plt.show()

In [ ]:
plt.figure(figsize=(10,10))
plt.plot(np.arange(0,1,0.0001),np.arange(0,1,0.0001),"g")
plt.plot(FPR_array,TPR_array,"k")
plt.xlabel("False Positive Rate")
plt.ylabel("True Positive Rate")
plt.title("ROC curve LOGISTIC FISHER NO-BACKTRACK")

In [ ]:
log_difference_list={}
for col in Actuator_list:
    prediction_matrix=dataNorm_reg_adpt.drop([col],axis=1).values
    dupa=[]
    for x in prediction_matrix:
        m=d_log["classifier_{0}".format(col)].predict(np.array(x).reshape(1,-1))
        dupa.append(float(m))
    dupa=np.array(dupa)
    difference=dupa-np.array(dataNorm_reg_adpt[col])
    log_difference_list["diff_{0}".format(col)]=difference

In [ ]:
log_attack_difference_list={}
for col in Actuator_list:
    prediction_matrix=dataAtck_reg_adpt.drop([col],axis=1).values
    dupa=[]
    for x in prediction_matrix:
        m=d_log["classifier_{0}".format(col)].predict(np.array(x).reshape(1,-1))
        dupa.append(float(m))
    dupa=np.array(dupa)
    difference=dupa-np.array(dataAtck_reg_adpt[col])
    log_attack_difference_list["diff_{0}".format(col)]=difference

In [ ]:
zero_log_difference_list={}
for col in rest_actuator:
    difference=np.array(dataNorm_reg_adpt[col])-np.array(dataNorm_reg_adpt[col])
    zero_log_difference_list["diff_{0}".format(col)]=difference

In [ ]:
zero_log_attack_difference_list={}
for col in rest_actuator:
    difference=-np.array(dataAtck_reg_adpt[col])
    zero_log_attack_difference_list["diff_{0}".format(col)]=difference

In [ ]:
z= {**log_difference_list, **zero_log_difference_list}

In [ ]:
m= {**log_attack_difference_list, **zero_log_attack_difference_list}


In [ ]:
import pickle

path="/Pickles"
pickleFolder=os.getcwd()+path

#with open('z.pickle', 'wb') as handle:
#    pickle.dump(z, handle, protocol=pickle.HIGHEST_PROTOCOL)
with open(pickleFolder+'/z.pickle', 'rb') as handle:
    gg = pickle.load(handle)
#with open('m.pickle', 'wb') as handle:
#    pickle.dump(m, handle, protocol=pickle.HIGHEST_PROTOCOL)
with open(pickleFolder+'/m.pickle', 'rb') as handle:
    ff = pickle.load(handle)

# start here

In [ ]:
#import pickle
#with open('log_attack_difference.pickle', 'wb') as handle:
#    pickle.dump(log_attack_difference_list, handle, protocol=pickle.HIGHEST_PROTOCOL)
with open('log_attack_difference.pickle', 'rb') as handle:
    b = pickle.load(handle)
log_attack_difference=b

In [ ]:
histogram_list={}
for col in actuator_list:
    plt.figure(figsize=(20,10))
    num_bins=5
    n, bins, patches = plt.hist(gg["diff_{0}".format(col)], bins=np.arange(-2,4,1))
    plt.title("Normal Histogram %s" % col)
    histogram_list["norm_hist_{0}".format(col)]=n
    plt.show()
    plt.figure(figsize=(20,10))
    n, bins, patches = plt.hist(ff["diff_{0}".format(col)], bins=np.arange(-2,4,1))
    plt.title("Attack Histogram %s" % col)
    histogram_list["atck_hist_{0}".format(col)]=n
    plt.show()

this is just for training

In [ ]:
with open('histogram_list.pickle', 'wb') as handle:
    pickle.dump(histogram_list, handle, protocol=pickle.HIGHEST_PROTOCOL)
#with open('histogram_list.pickle', 'rb') as handle:
#    b = pickle.load(handle)

to predict probabilities

In [ ]:
normalIndex=list(dataAtck.iloc[list(dataAtck["Normal/Attack"]=="Normal")].index)
attackIndex=list(dataAtck.iloc[list(dataAtck["Normal/Attack"]!="Normal")].index)

In [ ]:
import pickle
#with open('difference.pickle', 'wb') as handle:
#    pickle.dump(difference_list, handle, protocol=pickle.HIGHEST_PROTOCOL)
with open('difference.pickle', 'rb') as handle:
    b = pickle.load(handle)
difference_list=b

In [ ]:
import pickle
with open('attack_difference.pickle', 'wb') as handle:
    pickle.dump(attack_difference_list, handle, protocol=pickle.HIGHEST_PROTOCOL)
with open('attack_difference.pickle', 'rb') as handle:
    c = pickle.load(handle)
attack_difference_list=c

In [ ]:
hist_data=pd.DataFrame([],columns=actuator_list)
for col in actuator_list:
    normalisedHist=histogram_list["norm_hist_{0}".format(col)]/sum(histogram_list["norm_hist_{0}".format(col)])
    hist_data[col]=normalisedHist

In [ ]:
pvalue_data=hist_data.copy()
for col in pvalue_data.columns:
    for i in range(0,len(hist_data)):
        a=hist_data[col][i]
        truthVec=(hist_data[col]<=a)
        pvalue_data[col][i]=sum(np.multiply(hist_data[col],truthVec))

In [ ]:
attack_diff_data=pd.DataFrame([],columns=actuator_list).copy()
for col in attack_diff_data:
    attack_diff_data[col]=ff["diff_{0}".format(col)]

In [ ]:
bin_ref_data=(attack_diff_data+2).copy()
attack_pvalues=bin_ref_data.copy()
for col in bin_ref_data:
    print(col)
    L=np.array([math.floor(x) for x in bin_ref_data[col]])
    for i in range(0,len(bin_ref_data)):
        try:
            if pvalue_data[col][L[i]]!=0:
                attack_pvalues.loc[:,col]=pvalue_data[col][L[i]]
            else:
                attack_pvalues.loc[:,col]=0.000001
        except:
            attack_pvalues.loc[:,col]=0.000001

In [ ]:
#attack_pvalues.to_pickle("./logpvalues.pkl")
attack_pvalues = pd.read_pickle("./logpvalues.pkl")

In [ ]:
attack_pvalues

In [ ]:
combinedValue=np.array([])
for x in attack_pvalues.values:
    combinedValue=np.hstack((combinedValue,sum(np.log(x))))

In [ ]:
attack_pvalues["Combined"]=-2*combinedValue

In [ ]:
plt.figure(figsize=(20,10))
#for m in attack_start_times:
#    plt.axvline(x=m,color="r",linestyle="-")
plt.plot(attack_pvalues["Combined"],".b")
plt.show()

In [ ]:
h=open("/home/sharding/Documents/SummerProject/Data/AttackList_Indexed.csv","r+")
file_atcklist=csv.reader(h)
top=next(file_atcklist)
attack_list=pd.DataFrame(file_atcklist)

In [ ]:
base=time.strptime('12/28/2015 10:00:00', '%m/%d/%Y %H:%M:%S')
attack_start_times=[]
for t in range(0,len(attack_list)):
    attack_start_times.append(int(time.mktime(time.strptime(attack_list.iloc[t][1], '%m/%d/%Y %H:%M:%S'))-time.mktime(base)))

In [ ]:
def time_to_detect(t):
    for T in range(t,t+1000):
        if sum(np.array(attack_significance_data.iloc[T])!=0)>=5:
            return(T-t)
    return("No Error Detected")

In [ ]:
detection_times=[]
for k in attack_start_times:
    detection_times.append(time_to_detect(k))

# Here the confusion matrix is missing

In [ ]:
detection_times

In [ ]:
a=0
for x in attack_significance_data.values:
    b=(sum(np.array(x)!=0)>=7)
    a=a+b

In [ ]:
a

In [ ]:
attack_significance_data["number of significant features"]=np.array([15-sum(np.array(x)==0) for x in attack_significance_data.values])

In [ ]:
attack_significance_data

In [ ]:
%matplotlib qt
plt.figure(figsize=(200,100))
for m in attack_start_times:
    plt.axvline(x=m,color="r",linestyle="-")
plt.plot(list(attack_significance_data["number of significant features"]))
plt.show()

In [ ]:
from sklearn.datasets import load_iris
from sklearn.linear_model import LogisticRegression
#>>> X, y = load_iris(return_X_y=True)
y=dataNorm_reg_adpt["P101"]
X=dataNorm_reg_adpt.drop(["P101"],axis=1).copy().values
clf = LogisticRegression(random_state=0, solver='lbfgs',multi_class='multinomial').fit(X, y)
clf.predict(X[:2, :])

#>>> clf.predict_proba(X[:2, :]) 
#array([[9.8...e-01, 1.8...e-02, 1.4...e-08],
#       [9.7...e-01, 2.8...e-02, ...e-08]])
#>>> clf.score(X, y)
#0.97...

In [ ]:
clf.predict_proba(X[:2]) 